<a href="https://colab.research.google.com/github/ykitaguchi77/CorneAI/blob/main/50_knock_csv_manipulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**「前原の100本ノック」クラス枚数確認**

In [1]:
from google.colab import drive
drive.mount("/content/drive")


Mounted at /content/drive


In [64]:
import pandas as pd

# 100問_正解リスト.csvファイルからデータフレームを作成する
ans_df = pd.read_csv('/content/drive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の100問/100問_正解リスト.csv')

# 100問ノック_スマホと細隙灯区分.csvファイルからデータフレームを作成する
knock_df = pd.read_csv('/content/drive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の100問/100問ノック_スマホと細隙灯区分.csv')

In [58]:
ans_df

,disease,classification,number
0,翼状片,腫瘍,1
1,瘢痕,瘢痕,2
2,白内障,水晶体混濁,3
3,緑内障発作,緑内障発作,4
4,真菌,感染,5
...,...,...,...
95,白内障,水晶体混濁,96
96,水疱性角膜症,水疱性角膜症,97
97,緑内障発作,緑内障発作,98
98,真菌,感染,99


In [59]:
knock_df

,slit,smartphone
0,1,3
1,2,4
2,5,6
3,7,9
4,8,10
5,12,11
6,78,13
7,65,14
8,18,15
9,74,16


In [65]:
import pandas as pd

# "device"の行をdf_answerに追加
ans_df['device'] = ""

# "slit"と"smartphone"の列を取得
slit = knock_df["slit"].tolist()
smartphone = knock_df["smartphone"].tolist()

# "device"列に"slit"と"smartphone"を入力
for i in range(1,101):
    if i in slit:
        ans_df.loc[ans_df["number"]==i, "device"] = "slit"
    elif i in smartphone:
        ans_df.loc[ans_df["number"]==i, "device"] = "smartphone"

# インデックスをリセット
ans_df = ans_df.reset_index(drop=True)
ans_df

,disease,classification,number,device
0,翼状片,腫瘍,1,slit
1,瘢痕,瘢痕,2,slit
2,白内障,水晶体混濁,3,smartphone
3,緑内障発作,緑内障発作,4,smartphone
4,真菌,感染,5,slit
...,...,...,...,...
95,白内障,水晶体混濁,96,smartphone
96,水疱性角膜症,水疱性角膜症,97,smartphone
97,緑内障発作,緑内障発作,98,slit
98,真菌,感染,99,smartphone


In [73]:
smartphone_df = ans_df[ans_df['device'] == 'smartphone']
smartphone_classification_counts = smartphone_df['classification'].value_counts()
print("前原100本ノック_スマホ画像の各クラス枚数一覧")
print(smartphone_classification_counts)

前原100本ノック_スマホ画像の各クラス枚数一覧
正常        7
沈着        7
水晶体混濁     6
感染        6
腫瘍        6
水疱性角膜症    5
非感染       5
緑内障発作     4
瘢痕        4
Name: classification, dtype: int64


In [72]:
slit_df = ans_df[ans_df['device'] == 'slit']
slit_classification_counts = slit_df['classification'].value_counts()
print("前原100本ノック_スリット画像の各クラス枚数一覧")
print(slit_classification_counts)

前原100本ノック_スリット画像の各クラス枚数一覧
正常        7
沈着        7
腫瘍        6
感染        6
水晶体混濁     6
水疱性角膜症    5
非感染       5
瘢痕        4
緑内障発作     4
Name: classification, dtype: int64


In [48]:
ans_df


,disease,classification,number,device
0,翼状片,腫瘍,1,
1,瘢痕,瘢痕,2,
2,白内障,水晶体混濁,3,
3,緑内障発作,緑内障発作,4,
4,真菌,感染,5,
...,...,...,...,...
95,白内障,水晶体混濁,96,
96,水疱性角膜症,水疱性角膜症,97,
97,緑内障発作,緑内障発作,98,
98,真菌,感染,99,


In [44]:
ans_df[ans_df["number"]==1]

,disease,classification,number,device
0,翼状片,腫瘍,1,


In [32]:
ans_df

,disease,classification,number,device
0,翼状片,腫瘍,1,
1,瘢痕,瘢痕,2,slit
2,白内障,水晶体混濁,3,slit
3,緑内障発作,緑内障発作,4,smartphone
4,真菌,感染,5,smartphone
...,...,...,...,...
95,白内障,水晶体混濁,96,slit
96,水疱性角膜症,水疱性角膜症,97,smartphone
97,緑内障発作,緑内障発作,98,smartphone
98,真菌,感染,99,slit


In [74]:
ans_df.to_csv("/content/drive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の100問/100問_正解リスト_2.csv")